In [1]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

In [68]:
# df = pl.read_excel('../data/5206006_Industry_GVA.xlsx', sheet_name='Data1', has_header=True, read_options={'header_row': 9}, infer_schema_length=10000).select([
df = pl.read_excel('../data/5204005_GVA_by_Industry.xlsx', sheet_name='Data1', has_header=True, read_options={'header_row': 9}, infer_schema_length=10000).select([
    pl.col('Series ID').alias('Quarter'),
    pl.col('A3347242V').alias('Agriculture, forestry and fishing (A)'),
    pl.col('A3347281K').alias('Mining (B)'),
    pl.col('A3347318C').alias('Manufacturing (C)'),
    pl.col('A3348082L').alias('Electricity, gas, water and waste services (D)'),
    pl.col('A3348119F').alias('Construction (E)'),
    pl.col('A3348153K').alias('Wholesale trade (F)'),
    pl.col('A3348187J').alias('Retail trade (G)'),
    pl.col('A3348221A').alias('Accommodation and food services (H)'),
    pl.col('A3348255X').alias('Transport, postal and warehousing (I)'),
    pl.col('A3348294R').alias('Information media and telecommunications (J)'),
    pl.col('A3348328A').alias('Financial and insurance services (K)'),
    pl.col('A3348362F').alias('Rental, hiring and real estate services (L)'),
    pl.col('A3348396C').alias('Professional, scientific and technical services (M)'),
    pl.col('A3348430W').alias('Administrative and support services (N)'),
    pl.col('A3346692K').alias('Public administration and safety (O)'),
    pl.col('A3346726W').alias('Education and training (P)'),
    pl.col('A3346760A').alias('Health care and social assistance (Q)'),
    pl.col('A3346796C').alias('Arts and recreation services (R)'),
    pl.col('A3346831X').alias('Other services (S)'),
]).unpivot(index='Quarter', variable_name='Sector', value_name='GVA').filter(pl.col('GVA').is_not_null()).with_columns([
    pl.col('Quarter').dt.offset_by('6mo').dt.year().alias('FY'),
    pl.col('Sector').str.extract(r'\((\w)\)').alias('Code'),
]).group_by(['FY', 'Sector', 'Code']).agg([
    pl.col('GVA').sum()
]).sort(['Code', 'FY'], descending=[False, False])

In [69]:
colour_map = {k:v for k,v in zip(
    df.select(['Code', 'Sector']).sort('Code')['Sector'].unique(maintain_order=True).to_list(),
    px.colors.qualitative.Dark24
)}

In [73]:
from plotly.offline import plot

In [82]:
fig = px.pie(df.filter(pl.col('FY') == 2025), names='Sector', values='GVA', color_discrete_map=colour_map)
# fig.update_layout(widt)
fig

In [78]:
fig.write_html('../docs/posts/20251130_value-added-per-sector_pie.html', include_plotlyjs='cdn')
# fig.write_html('../docs/posts/20251130_value-added-per-sector_pie.html')

In [72]:
fig = go.Figure()
for (sector,), sector_df in df.group_by('Sector', maintain_order=True):
    fig.add_trace(go.Scatter(
        x=sector_df['FY'],
        y=sector_df['GVA'],
        mode='lines',
        fill='tonexty',
        name=sector,
        stackgroup='one',
        legendgroup=sector,
        # groupnorm='fraction',
    ))
fig.show()

In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scatter"}, {"type": "domain"}]], subplot_titles=("GVA by Sector Over Time", "FY25 Distribution"))
for (sector,), sector_df in df.group_by('Sector', maintain_order=True):
    fig.add_trace(go.Scatter(
        x=sector_df['FY'],
        y=sector_df['GVA'],
        mode='lines',
        fill='tonexty',
        name=sector,
        stackgroup='one',
        legendgroup=sector,
        groupnorm='fraction',
    ), row=1, col=1)

fig.add_trace(go.Pie(
    labels=df.filter(pl.col('FY') == 2025)['Sector'],
    values=df.filter(pl.col('FY') == 2025)['GVA'],
    name='2025',
    hole=0.4,
    legendgroup=
), row=1, col=2)
fig.show()

In [ ]:
px.area